# Project 5: Logic Gates and Gate Nets
Outside of residual knowledge from 201, I got help from Ryan Hankee who suggested I do this sim in OOP-style rather than my normal code throwup. I got a hand with the class structure (Logic -> Gate -> NAND/NOR/NOT). I implemented a verstatile simulation method using the class structure that will operate on both single and combinational gate simulations, defaulting to single gate sims.

In [26]:
from vpython import*
from queue import PriorityQueue

class Net:
    def __init__(self, driver, obj, time, on=0):
        self.value = None
        self.obj = obj
        self.driver = driver
        self.receivers = []
        self.on = on 
        self.time = time
        
    def set(self, value):
        if self.value == value:
            return 
        
        self.value = value
        
        if self.on:
            self.driver.evaluate()
            
        for c in self.receivers:
            c.set(value)

    def connect(self, inputs):
        if not isinstance(inputs, list):
            inputs = [inputs]
            
        for input in inputs:
            self.receivers.append(input)


class Logic:
    def __init__(self, name):
        self.name = name

    def evaluate(self):
        return

class Gate(Logic):        
    def __init__(self, name, obj, delay):
        Logic.__init__(self, name)
        self.obj = obj
        self.A = Net(self, 0, time=0, on=1)
        self.B = Net(self, 1, time=0, on=1)
        self.C = Net(self, 2, time=delay, on=1)

class Not(Gate):     
    def __init__(self, name, obj, delay):
        Gate.__init__(self, name)
        self.obj = obj
        self.A = Net(self, 0, time=0, on=1)
        self.B = Net(self, 2, time=delay, on=1)

    def evaluate(self):
        result = not self.A.value
        self.B.set(int(result))

class Nor(Gate):         
    def __init__(self, name, obj):
        Gate.__init__(self, name, obj, 1)

    def evaluate(self):
        result = not(self.A.value or self.B.value)
        self.C.set(int(result))
   
class Nand(Gate):       
    def __init__(self, name, obj):
        Gate.__init__(self, name, obj, 1)
        
    def evaluate(self):
        result = not(self.A.value and self.B.value)
        self.C.set(int(result))

class Event:
    
    def __init__(self, net, t):
        self.obj = net 
        self.value = net.value 
        self.time = net.time + t
        self.type = net.obj
        
    def __lessEq__(self, other): return self.time <= other.time
    
    def __less__(self, other): return self.time < other.time
        
    def __eq__(self, other): return self.time == other.time
    
    def __great__(self, other): return self.time > other.time
    
    def __greatEq__(self, other): return self.time >= other.time
        
   
def setInput(gate, a, b):
    gate.A.set(a)
    gate.B.set(b)

def netUpdate(gate, input_values, index):
    a,b = input_values[index]
    setInput(gate, a, b)

def getEvent(eq1, eq2, eq3):
    e1 = eq1.get()
    e2 = eq2.get()
    e3 = eq3.get()
    return e1, e2, e3

def eventInit(gate, values, time):
    event_q1 = PriorityQueue()
    event_q2 = PriorityQueue()
    event_q3 = PriorityQueue()

    netUpdate(gate, values, 0)

    e1 = Event(gate.A, time)
    e2 = Event(gate.B, time)
    e3 = Event(gate.C, time)

    event_q1.put(e1)
    event_q2.put(e2)
    event_q3.put(e3)
    return event_q1, event_q2, event_q3

def eventNetInit(gate, input1, input2, values, time):
    event_q1 = PriorityQueue()
    event_q2 = PriorityQueue()
    event_q3 = PriorityQueue()

    netUpdate(input1, values, 0)
    netUpdate(input2, values, 0)

    e1 = Event(gate.A, time)
    e2 = Event(gate.B, time)
    e3 = Event(gate.C, time)

    event_q1.put(e1)
    event_q2.put(e2)
    event_q3.put(e3)

    return event_q1, event_q2, event_q3

def sim(t, clock, setup, values, gate, gate2, g1, g2, g3, oneOrTwo):
    index = 1
    e1, e2, e3 = setup
    while not (e3.empty()):
        event1, event2, event3 = getEvent(e1, e2, e3)
        t += event3.time - clock
        clock = t

        g1.plot(clock, event1.value)
        g2.plot(clock, event2.value)
        g3.plot(clock, event3.value)


        if index < len(values):
            if oneOrTwo == 2:
                netUpdate(gate, values, index)
                netUpdate(gate2, values, index)
            else:
                netUpdate(gate, values, index)

            index += 1

            next_event1 = Event(event1.obj, clock)
            next_event2 = Event(event2.obj, clock)
            next_event3 = Event(event3.obj, clock)
            e1.put(next_event1)
            e2.put(next_event2)
            e3.put(next_event3)
            g1.plot(clock, next_event1.value)
            g2.plot(clock, next_event2.value)
            g3.plot(clock, next_event3.value)



# NAND verification

In [31]:
scene = canvas()
t = 0
clock = 0
nand = Nand('nand1', 0)
truth_values = [(1,1), (0,0), (0,0), (0,1), (1,0), (1,1), (0,0), (0,1), (1,0), (1,1), (0,0), (0,1), (1,0), (1,1)]
g1 = graph(title="Input Signal A", xtitle='time', ytitle='boolean signal', fast=False, width=1000)
g2 = graph(title="Input Signal B", xtitle='time', ytitle='boolean signal', fast=False, width=1000)
g3 = graph(title="Output Signal AB", xtitle='time', ytitle='boolean signal', fast=False, width=1000)
netA = gcurve(graph=g1, color=color.red, width=5, markers=True, marker_color=color.black, label='netA')
netB = gcurve(graph=g2, color=color.green, width=5, markers=True, marker_color=color.black, label='netB')
netC = gcurve(graph=g3, color=color.blue, width=5, markers=True, marker_color=color.black, label='nand')

sim(t, clock, setup_event(nand, truth_values, t), truth_values, nand, nand, netA, netB, netC, 1)

<IPython.core.display.Javascript object>

# NOR Verification

In [30]:
scene = canvas()
t = 0
clock = 0
nor = Nor('nor1', 0)
truth_table = [(0,1), (0,1), (0,0), (0,1), (1,0), (1,1), (0,0), (0,1), (1,0), (1,1), (0,0), (0,1), (1,0), (1,1)]
g4 = graph(title="Input Signal A", xtitle='time', ytitle='boolean signal', fast=False, width=1000)
g5 = graph(title="Input Signal B", xtitle='time', ytitle='boolean signal', fast=False, width=1000)
g6 = graph(title="Output Signal AB", xtitle='time', ytitle='boolean signal', fast=False, width=1000)
netD = gcurve(graph=g4, color=color.red, width=5, markers=True, marker_color=color.black, label='netA')
netE = gcurve(graph=g5, color=color.green, width=5, markers=True, marker_color=color.black, label='netB')
netF = gcurve(graph=g6, color=color.blue, width=5, markers=True, marker_color=color.black, label='nor')

sim(t, clock, setup_event(nor, truth_table, t), truth_table, nor, nor, netD, netE, netF, 1)

<IPython.core.display.Javascript object>

# Combinational NAND

In [28]:
scene = canvas()
t = 0
clock = 0
nand1 = Nand('nand1', 0)
nand2 = Nand('nand2', 1)
nand3 = Nand('nand3', 2)
nand1.C.connect([nand3.A])
nand2.C.connect([nand3.B])
g7 = graph(title="NAND1 Output", xtitle='time', ytitle='value', fast=False, width=800)
g8 = graph(title="NAND2 Output", xtitle='time', ytitle='value', fast=False, width=800)
g9 = graph(title="Network Output", xtitle='time', ytitle='value', fast=False, width=900)
netG = gcurve(graph=g7, color=color.blue, width=5, markers=True, marker_color=color.black, label='net1')
netH = gcurve(graph=g8, color=color.yellow, width=5, markers=True, marker_color=color.black, label='net2')
netI = gcurve(graph=g9, color=color.red, width=5, markers=True, marker_color=color.black, label='Nand3')

sim(t, clock, setup_Net_event(nand3, nand1, nand2, truth_values, t), truth_values, nand1, nand2, netG, netH, netI, 2)

<IPython.core.display.Javascript object>

# Combinational NOR

In [29]:
scene = canvas()
t = 0
clock = 0
nor1 = Nor('nor1', 0)
nor2 = Nor('nor2', 1)
nor3 = Nor('nor3', 2)
nor1.C.connect([nor3.A])
nor2.C.connect([nor3.B])
g10 = graph(title="NOR1 Output", xtitle='time', ytitle='value', fast=False, width=800)
g11 = graph(title="NOR2 Output", xtitle='time', ytitle='value', fast=False, width=800)
g12 = graph(title="Network Output", xtitle='time', ytitle='value', fast=False, width=900)
netJ = gcurve(graph=g10, color=color.blue, width=5, markers=True, marker_color=color.black, label='net1')
netK = gcurve(graph=g11, color=color.yellow, width=5, markers=True, marker_color=color.black, label='net2')
netL = gcurve(graph=g12, color=color.red, width=5, markers=True, marker_color=color.black, label='NOR3')

sim(t, clock, setup_Net_event(nor3, nor1, nor2, truth_values, t), truth_values, nor1, nor2, netJ, netK, netL, 2)

<IPython.core.display.Javascript object>

# D Latch (NAND)

In [ ]:
clk = 0
systemClk = 0
clock_values = [0,1,1,0,1,1,0,1,1,0,0]
data_values =  [0,0,1,0,0,1,0,0,1,1,0]

nand1 = Nand('nand1_S', 0)
nand2 = Nand('nand2_R', 1)
nand3 = Nand('nand3_Q', 2)
nand4 = Nand('nand4_Qb', 3)
not1 = Not('not1', 0, 1)

nand1.B.connect([nand2.A])
nand1.C.connect([nand3.A])
nand1.A.connect([not1.A])
not1.B.connect([nand2.B])
nand4.C.connect([nand3.B])
nand2.C.connect([nand4.B])
nand3.C.connect([nand4.A])

nand1.A.set(data_values[0])
nand1.B.set(clock_values[0])

## Not sure where to go from here ##